Initial setup

In [1]:
import cartopy.crs as ccrs
import earthaccess
import h5netcdf
import matplotlib.pyplot as plt
import numpy as numpy
import xarray as xr
from datetime import date, timedelta
import geopandas as pd
from shapely.geometry import shape
import datetime
import os
import math
import rasterio 
import rasterio.plot
import pyproj
import csv

In [14]:
auth = earthaccess.login(persist = True)

##Parameter for CA Palisades Fire
latmin = 33.78
latmax = 34.07
lonmin = -118.8
lonmax = -118.4

timePeriod = ("2025-01-01", "2025-03-08")
AOI = (-118.8, 33.78, -118.4, 34.07)
latbounds = [34.07, 33.78]
lonbounds = [-118.8, -118.37]

location = "PFDF_Data/PalisadesFire2025/"

def ncRasterAverage(latmin, latmax, lonmin, lonmax, dataset):
    total = 0
    count = 0
    avg = 0

    for i in range(1, len(dataset["lat"]) - 1):
        if (dataset.lat[i].item(0) > latmin):
            if (dataset.lat[i].item(0) < latmax):
                for j in range(1, len(dataset["lon"])- 1):
                    if (dataset.lon[j].item(0) > lonmin):
                        if (dataset.lon[j].item(0) < lonmax):
                            value = dataset.sel(lat = dataset.lat[i],lon = dataset.lon[j]).item(0)
                            if (not math.isnan(value)):
                                value = float(value)
                                count += 1
                                print(f'lat: {dataset.lat[i].item(0)}, lon: {dataset.lon[j].item(0):}: {value}')
                                total = total + value
                                #print(total)
                            
    if (count == 0):
        return 0
    
    else: 
        avg = total/count
        return avg



Copernicus access

In [18]:
"""f = open("PFDF_Data/Credentials/Copernicus.txt")
user_Copernicus = f.readline()
pass_Copernicus = f.readline()

ft = "POLYGON((0 0, 0 1, 1 1, 1 0, 0 0))"
data_collection = "SENTINEL-2"

today =  date.today()
today_string = today.strftime("%Y-%m-%d")
yesterday = today - timedelta(days=1)
yesterday_string = yesterday.strftime("%Y-%m-%d")

def get_keycloak(username: str, password: str) -> str:
    data = {
        "client_id": "cdse-public",
        "username": user_Copernicus,
        "password": pass_Copernicus,
        "grant_type": "password"
    }

    try: 
        r = requests.post(
            "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
            data = data,
        )
        r.raise_for_status()
    except Exception as e:
        raise Exceptions(
            f"Keycloak token creation failed. Response from the server was: {r.json()}"
        )
    return r.json()["access_token"]"""

Burn Area identification

In [ ]:
""" json_ = requests.get(
    f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq '{data_collection}' and OData.CSC.Intersects(area=geography'SRID=4326;{ft}') and ContentDate/Start gt {yesterday_string}T00:00:00.000Z and ContentDate/Start lt {today_string}T00:00:00.000Z&$count=True&$top=1000"
).json()
p = pd.DataFrame.from_dict(json_["value"])
if p.shape[0] > 0 : 
    p["geometry"] = p["GeoFootprint"].apply(shape)
    productDF = gpd.GeoDataFrame(p).set_geometry("geometry")
    productDF = productDF[~productDF["Name"].str.contains("L1C")]
    print(f"total L2A tiles found {len(productDF)}")
    productDF["identifier"] = productDF["Name"].srt.split(".").str[0]
    allfeat = len(productDF)

    if allfeat == 0:
        print(f"No tiles found for {today_string}")
    else: 
        for index, feat in enumerate(productDF.iterfeatures()):
            try:
                session = requests.Session()
                keycloak_token = get_keycloak(user_Copernicus, pass_Copernicus)

                session.headers.update({"Authorization": f"Bearer {keycloak_token}"})
                url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products({feat['properties']['Id']})/$value"
                response = session.get(url, allow_redirects = False)
                while response.status_code in (301, 302, 303, 307):
                    url = response.headers["Location"]
                    response = session.get
                print(feat["properties"]["Id"])
                file = session.get(url, verify=False, allow_redirects=True)

                with open(
                    f"location/to/save/{feat['properties']['identifier']}.zip", #location to save zip from copernicus 
                    "wb",
                ) as p:
                    print(feat["properties"]["Name"])
                    p.write(file.content)
            except:
                print("problem with server")
else : # If no tiles found for given date range and AOI
    print('no data found') """



PACE Chlorophyll-a products

In [3]:
timePeriod = ("2025-01-01", "2025-03-08")
AOI = (-118.8, 33.78, -118.4, 34.07)
#precipitationDays = ["20250126", "20250127", "20250128", "20250129", 
#              "20250206", "20250207", "20250208", "20250209",
#              "20250213", "20250214", "20250215", "20250216"]
#pathRoot = "PFDF_Data/California2025/"

latbounds = [34.07, 33.78]
lonbounds = [-118.8, -118.37]

In [7]:
results = earthaccess.search_datasets(instrument = "oci")

for item in results:
    summary = item.summary()
    print(summary["short-name"])

PACE_OCI_L2_AOP_NRT
PACE_OCI_L2_BGC_NRT
PACE_OCI_L0_SCI
PACE_OCI_L1A_SCI
PACE_OCI_L1B_SCI
PACE_OCI_L1B_SCI
PACE_OCI_L1C_SCI
PACE_OCI_L1C_SCI
PACE_OCI_L2_AOP
PACE_OCI_L2_AOP_NRT
PACE_OCI_L2_BGC
PACE_OCI_L2_BGC_NRT
PACE_OCI_L2_CLOUD
PACE_OCI_L2_CLOUD_MASK
PACE_OCI_L2_CLOUD_MASK_NRT
PACE_OCI_L2_CLOUD_NRT
PACE_OCI_L2_IOP
PACE_OCI_L2_IOP_NRT
PACE_OCI_L2_IOP_NRT
PACE_OCI_L2_LANDVI
PACE_OCI_L2_LANDVI_NRT
PACE_OCI_L2_PAR
PACE_OCI_L2_PAR_NRT
PACE_OCI_L2_PAR_NRT
PACE_OCI_L2_SFREFL
PACE_OCI_L2_SFREFL_NRT
PACE_OCI_L2_SFREFL_NRT
PACE_OCI_L3B_AVW
PACE_OCI_L3B_AVW_NRT
PACE_OCI_L3B_AVW_NRT
PACE_OCI_L3B_CARBON
PACE_OCI_L3B_CARBON_NRT
PACE_OCI_L3B_CHL
PACE_OCI_L3B_CHL_NRT
PACE_OCI_L3B_CHL_NRT
PACE_OCI_L3B_FLH
PACE_OCI_L3B_FLH_NRT
PACE_OCI_L3B_IOP
PACE_OCI_L3B_IOP_NRT
PACE_OCI_L3B_IOP_NRT
PACE_OCI_L3B_KD
PACE_OCI_L3B_KD_NRT
PACE_OCI_L3B_KD_NRT
PACE_OCI_L3B_LANDVI
PACE_OCI_L3B_LANDVI_NRT
PACE_OCI_L3B_PAR
PACE_OCI_L3B_PAR_NRT
PACE_OCI_L3B_PAR_NRT
PACE_OCI_L3B_POC
PACE_OCI_L3B_POC_NRT
PACE_OCI_L3B_POC_NRT
P

In [ ]:
tspan = timePeriod
bbox = AOI
clouds = (0,50)

results = earthaccess.search_data(
    short_name = "PACE_OCI_L3M_CHL_NRT",
    temporal = tspan,
    bounding_box = bbox
)

paths = earthaccess.open(results)
paths 

chlDayData = []
#pathString = "WaterQuality/PACE/Chl_a/"

for file in paths:
    testxr = xr.open_dataset(file)
    if ("DAY" in testxr.product_name):
        if ("4km" in testxr.product_name):
            if ("V3" in testxr.product_name):
                chlDayData.append(testxr)
                print(f'{testxr.product_name} added!')
                #testxr.to_netcdf(path = (pathRoot + pathString + testxr.product_name))
                #print(f'{testxr.product_name} downloaded successfully.')

In [8]:

def ncRasterAverage(latmin, latmax, lonmin, lonmax, dataset):
    total = 0
    count = 0
    avg = 0

    for i in range(1, len(dataset["lat"]) - 1):
        if (dataset.lat[i].item(0) > latmin):
            if (dataset.lat[i].item(0) < latmax):
                for j in range(1, len(dataset["lon"])- 1):
                    if (dataset.lon[j].item(0) > lonmin):
                        if (dataset.lon[j].item(0) < lonmax):
                            value = dataset.sel(lat = dataset.lat[i],lon = dataset.lon[j]).item(0)
                            if (not math.isnan(value)):
                                value = float(value)
                                count += 1
                                print(f'lat: {dataset.lat[i].item(0)}, lon: {dataset.lon[j].item(0):}: {value}')
                                total = total + value
                                #print(total)
                            
    if (count == 0):
        return 0
    
    else: 
        avg = total/count
        return avg

In [9]:
chlAverages = []

for file in chlDayData:
    if (ncRasterAverage(latmin, latmax, lonmin, lonmax, file["chlor_a"]) > 0):
        chlAverages.append([file.product_name, ncRasterAverage(latmin, latmax, lonmin, lonmax, file["chlor_a"])])

chlAverages

In [93]:
chlRecord = []
for day in chlAverages:
    chlRecord.append([day[0][9:17], day[1]])
chlRecord

chlPath = 'WaterQuality/PACE/Chl_a/Chlorophyll_a'

with open(location + chlPath, 'w') as myFile: 
    wr = csv.writer(myFile)
    for record in chlRecord:
        wr.writerow(record)


**PACE POC**

In [20]:
results = earthaccess.search_data(
    short_name = "PACE_OCI_L3M_POC_NRT",
    temporal = timePeriod,
    bounding_box = AOI
)

paths = earthaccess.open(results)

pacePOC = []

for file in paths:
    testxr = xr.open_dataset(file)
    if ("DAY" in testxr.product_name):
        if ("4km" in testxr.product_name):
            pacePOC.append(testxr)
            print(f'{testxr.product_name} added!')

QUEUEING TASKS | :   0%|          | 0/224 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/224 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/224 [00:00<?, ?it/s]

PACE_OCI.20250101.L3m.DAY.POC.V2_0.poc.4km.NRT.nc added!
PACE_OCI.20250101.L3m.DAY.POC.V3_0.poc.4km.NRT.nc added!
PACE_OCI.20250103.L3m.DAY.POC.V2_0.poc.4km.NRT.nc added!
PACE_OCI.20250103.L3m.DAY.POC.V3_0.poc.4km.NRT.nc added!
PACE_OCI.20250104.L3m.DAY.POC.V2_0.poc.4km.NRT.nc added!
PACE_OCI.20250104.L3m.DAY.POC.V3_0.poc.4km.NRT.nc added!
PACE_OCI.20250105.L3m.DAY.POC.V2_0.poc.4km.NRT.nc added!
PACE_OCI.20250105.L3m.DAY.POC.V3_0.poc.4km.NRT.nc added!
PACE_OCI.20250106.L3m.DAY.POC.V2_0.poc.4km.NRT.nc added!
PACE_OCI.20250106.L3m.DAY.POC.V3_0.poc.4km.NRT.nc added!
PACE_OCI.20250107.L3m.DAY.POC.V2_0.poc.4km.NRT.nc added!
PACE_OCI.20250107.L3m.DAY.POC.V3_0.poc.4km.NRT.nc added!
PACE_OCI.20250108.L3m.DAY.POC.V2_0.poc.4km.NRT.nc added!
PACE_OCI.20250108.L3m.DAY.POC.V3_0.poc.4km.NRT.nc added!
PACE_OCI.20250109.L3m.DAY.POC.V2_0.poc.4km.NRT.nc added!
PACE_OCI.20250109.L3m.DAY.POC.V3_0.poc.4km.NRT.nc added!
PACE_OCI.20250110.L3m.DAY.POC.V2_0.poc.4km.NRT.nc added!
PACE_OCI.20250110.L3m.DAY.POC.V

In [22]:
pacePOCavg = []

for file in pacePOC:
    avg = ncRasterAverage(latmin, latmax, lonmin, lonmax, file["poc"])
    if (avg > 0):
        pacePOCavg.append([file.product_name, avg])
        print(f'{file.product_name}: {avg} mg/m3 ')

pocPath = 'WaterQuality/PACE/POC/ParticulateOrganicCarbon'

for record in pacePOCavg:
    with open(location + pocPath, 'w') as myFile: 
        wr = csv.writer(myFile)
        for record in pacePOCavg:
            wr.writerow(record)
    

lat: 33.8125, lon: -118.77082824707031: 276.60009765625
lat: 33.8125, lon: -118.72916412353516: 276.60009765625
PACE_OCI.20250104.L3m.DAY.POC.V3_0.poc.4km.NRT.nc: 276.60009765625 mg/m3 
lat: 33.854164123535156, lon: -118.77082824707031: 345.0
lat: 33.854164123535156, lon: -118.72916412353516: 344.39990234375
lat: 33.854164123535156, lon: -118.6875: 344.39990234375
lat: 33.8125, lon: -118.6875: 324.39990234375
lat: 33.8125, lon: -118.64582824707031: 343.60009765625
lat: 33.8125, lon: -118.60416412353516: 312.19970703125
PACE_OCI.20250105.L3m.DAY.POC.V2_0.poc.4km.NRT.nc: 335.6665852864583 mg/m3 
lat: 33.9375, lon: -118.77082824707031: 304.0
lat: 33.9375, lon: -118.72916412353516: 304.0
lat: 33.9375, lon: -118.6875: 227.7998046875
lat: 33.89583206176758, lon: -118.77082824707031: 364.7998046875
lat: 33.89583206176758, lon: -118.72916412353516: 310.19970703125
lat: 33.89583206176758, lon: -118.6875: 257.39990234375
lat: 33.89583206176758, lon: -118.64582824707031: 350.0
lat: 33.89583206176

In [ ]:
pathString = "California2025/WaterQuality/PACE/Chl_a/"

for file in paths:
    testxr = xr.open_dataset(file)
    file.to_netcdf(path = (location + chlPath + testxr.product_name))

In [10]:
for file in chlDayData:
    fig = plt.figure()
    ax = plt.axes(projection = ccrs.PlateCarree())
    ax.coastlines()
    ax.gridlines(draw_labels = {"left": "y", "bottom": "x"})
    plot = file["chlor_a"].plot(x = "lon", y = "lat", cmap = "viridis", xlim = (-119, -118.3), ylim = (33.5, 34.2),
                                vmin = 0, vmax = 50)
    plt.savefig(pathString + "PalisadesFire_Focused_" + file.product_name + '.png')

NameError: name 'chlDayData' is not defined

**Sentinel-2 TSS products**

In [11]:
folder = location + "WaterQuality/Sentinel/TSS/"

tssAverages = []

for tssPath in os.listdir(folder):
    lonRange = numpy.arange(lonmin, lonmax, 0.01)
    latRange = numpy.arange(latmin, latmax, 0.01)

    tiff = rasterio.open(folder + tssPath)

    src_crs = "EPSG:4326"
    dst_crs = tiff.crs.to_proj4()

    count = 0
    total = 0

    for lon in lonRange:
        for lat in latRange:      
            coords = [[lat,lon]]
            transformer = pyproj.Transformer.from_crs("epsg:4326", tiff.crs)
            newCoords = [transformer.transform(x, y) for x, y in coords] 

            values = list(rasterio.sample.sample_gen(tiff, newCoords))
            if values[0] > 0:
                #print(f'{newCoords}: TSS: {values[0]}')
                count += 1
                total += values[0]
    
    if count > 0:
        avg = total/count
        tssAverages.append([str(tssPath)[11:19], avg.item(0)])

tssAverages

with open(location + folder + 'Palisades2025_tss', 'w') as myFile: 
    wr = csv.writer(myFile)
    for record in tssAverages:
        wr.writerow(record)


**PACE NDTI**

In [6]:
results = earthaccess.search_datasets(instrument="oci")

for item in results: 
    summary = item.summary()
    print(summary["short-name"])

PACE_OCI_L2_AOP_NRT
PACE_OCI_L2_BGC_NRT
PACE_OCI_L0_SCI
PACE_OCI_L1A_SCI
PACE_OCI_L1B_SCI
PACE_OCI_L1B_SCI
PACE_OCI_L1C_SCI
PACE_OCI_L1C_SCI
PACE_OCI_L2_AOP
PACE_OCI_L2_AOP_NRT
PACE_OCI_L2_BGC
PACE_OCI_L2_BGC_NRT
PACE_OCI_L2_CLOUD
PACE_OCI_L2_CLOUD_MASK
PACE_OCI_L2_CLOUD_MASK_NRT
PACE_OCI_L2_CLOUD_NRT
PACE_OCI_L2_IOP
PACE_OCI_L2_IOP_NRT
PACE_OCI_L2_IOP_NRT
PACE_OCI_L2_LANDVI
PACE_OCI_L2_LANDVI_NRT
PACE_OCI_L2_PAR
PACE_OCI_L2_PAR_NRT
PACE_OCI_L2_PAR_NRT
PACE_OCI_L2_SFREFL
PACE_OCI_L2_SFREFL_NRT
PACE_OCI_L2_SFREFL_NRT
PACE_OCI_L3B_AVW
PACE_OCI_L3B_AVW_NRT
PACE_OCI_L3B_AVW_NRT
PACE_OCI_L3B_CARBON
PACE_OCI_L3B_CARBON_NRT
PACE_OCI_L3B_CHL
PACE_OCI_L3B_CHL_NRT
PACE_OCI_L3B_CHL_NRT
PACE_OCI_L3B_FLH
PACE_OCI_L3B_FLH_NRT
PACE_OCI_L3B_IOP
PACE_OCI_L3B_IOP_NRT
PACE_OCI_L3B_IOP_NRT
PACE_OCI_L3B_KD
PACE_OCI_L3B_KD_NRT
PACE_OCI_L3B_KD_NRT
PACE_OCI_L3B_LANDVI
PACE_OCI_L3B_LANDVI_NRT
PACE_OCI_L3B_PAR
PACE_OCI_L3B_PAR_NRT
PACE_OCI_L3B_PAR_NRT
PACE_OCI_L3B_POC
PACE_OCI_L3B_POC_NRT
PACE_OCI_L3B_POC_NRT
P

In [7]:
tspan = timePeriod
bbox = AOI
clouds = (0,50)
#pathString = "WaterQuality/PACE/NDTI/"


results = earthaccess.search_data(
    short_name = "PACE_OCI_L2_AOP_NRT",
    temporal = tspan,
    bounding_box = bbox
)

paths = earthaccess.open(results)
l2aop = []

paths

#test = xr.open_dataset(paths[0])
#test

for file in paths:
    testxr = xr.open_dataset(file)
    if("V3" in testxr.product_name): 
        l2aop.append(testxr)
        print(f'{testxr.product_name} added!')

QUEUEING TASKS | :   0%|          | 0/113 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/113 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/113 [00:00<?, ?it/s]

PACE_OCI.20250104T202321.L2.OC_AOP.V3_0.NRT.nc added!
PACE_OCI.20250105T205806.L2.OC_AOP.V3_0.NRT.nc added!
PACE_OCI.20250106T195426.L2.OC_AOP.V3_0.NRT.nc added!
PACE_OCI.20250107T202910.L2.OC_AOP.V3_0.NRT.nc added!
PACE_OCI.20250108T210354.L2.OC_AOP.V3_0.NRT.nc added!
PACE_OCI.20250109T200019.L2.OC_AOP.V3_0.NRT.nc added!
PACE_OCI.20250110T203503.L2.OC_AOP.V3_0.NRT.nc added!
PACE_OCI.20250111T193627.L2.OC_AOP.V3_0.NRT.nc added!
PACE_OCI.20250111T210946.L2.OC_AOP.V3_0.NRT.nc added!
PACE_OCI.20250112T200611.L2.OC_AOP.V3_0.NRT.nc added!
PACE_OCI.20250113T204049.L2.OC_AOP.V3_0.NRT.nc added!
PACE_OCI.20250114T193713.L2.OC_AOP.V3_0.NRT.nc added!
PACE_OCI.20250114T211532.L2.OC_AOP.V3_0.NRT.nc added!
PACE_OCI.20250115T201156.L2.OC_AOP.V3_0.NRT.nc added!
PACE_OCI.20250116T204639.L2.OC_AOP.V3_0.NRT.nc added!
PACE_OCI.20250117T194303.L2.OC_AOP.V3_0.NRT.nc added!
PACE_OCI.20250117T212122.L2.OC_AOP.V3_0.NRT.nc added!
PACE_OCI.20250118T201746.L2.OC_AOP.V3_0.NRT.nc added!
PACE_OCI.20250119T205229.L2.

In [8]:
l2aop_nav = []

for item in paths:
    testxr = xr.open_dataset(item)
    if ("V3" in testxr.product_name): 
        print(testxr.product_name)
        print("Setting RRS...")
        dataGeoPhys = xr.open_dataset(item, group = "geophysical_data")
        dataRRS = dataGeoPhys["Rrs"]
        print("Retrieving coordinates...")
        dataNav = xr.open_dataset(item, group = "navigation_data")
        dataCoords = dataNav.set_coords(("longitude", "latitude"))
        print("Merging...")
        dataset = xr.merge((dataRRS, dataCoords.coords))
        dataset.attrs['name'] = testxr.product_name
        l2aop_nav.append(dataset)
        print("Done")



PACE_OCI.20250104T202321.L2.OC_AOP.V3_0.NRT.nc
Setting RRS...
Retrieving coordinates...
Merging...
Done
PACE_OCI.20250105T205806.L2.OC_AOP.V3_0.NRT.nc
Setting RRS...
Retrieving coordinates...
Merging...
Done
PACE_OCI.20250106T195426.L2.OC_AOP.V3_0.NRT.nc
Setting RRS...
Retrieving coordinates...
Merging...
Done
PACE_OCI.20250107T202910.L2.OC_AOP.V3_0.NRT.nc
Setting RRS...
Retrieving coordinates...
Merging...
Done
PACE_OCI.20250108T210354.L2.OC_AOP.V3_0.NRT.nc
Setting RRS...
Retrieving coordinates...
Merging...
Done
PACE_OCI.20250109T200019.L2.OC_AOP.V3_0.NRT.nc
Setting RRS...
Retrieving coordinates...
Merging...
Done
PACE_OCI.20250110T203503.L2.OC_AOP.V3_0.NRT.nc
Setting RRS...
Retrieving coordinates...
Merging...
Done
PACE_OCI.20250111T193627.L2.OC_AOP.V3_0.NRT.nc
Setting RRS...
Retrieving coordinates...
Merging...
Done
PACE_OCI.20250111T210946.L2.OC_AOP.V3_0.NRT.nc
Setting RRS...
Retrieving coordinates...
Merging...
Done
PACE_OCI.20250112T200611.L2.OC_AOP.V3_0.NRT.nc
Setting RRS...
Re

/Users/tfrankst/miniconda3/envs/pace/lib/python3.13/site-packages/xarray/backends/api.py:667: RuntimeWarning: 'h5netcdf' fails while guessing
  engine = plugins.guess_engine(filename_or_obj)


ValueError: found the following matches with the input file in xarray's IO backends: ['h5netcdf']. But their dependencies may not be installed, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html

In [27]:
l2aop_nav[0]

<xarray.Dataset> Size: 2GB
Dimensions:    (number_of_lines: 1710, pixels_per_line: 1272, wavelength_3d: 172)
Coordinates:
    longitude  (number_of_lines, pixels_per_line) float32 9MB ...
    latitude   (number_of_lines, pixels_per_line) float32 9MB ...
Dimensions without coordinates: number_of_lines, pixels_per_line, wavelength_3d
Data variables:
    Rrs        (number_of_lines, pixels_per_line, wavelength_3d) float32 1GB ...
Attributes:
    long_name:      Remote sensing reflectance
    units:          sr^-1
    standard_name:  surface_ratio_of_upwelling_radiance_emerging_from_sea_wat...
    valid_min:      -30000
    valid_max:      25000
    name:           PACE_OCI.20250104T202321.L2.OC_AOP.V3_0.NRT.nc

In [36]:
NDTIAverages = []

for dataset in l2aop_nav:
    print(dataset.name)
    
    lats = dataset.variables['latitude']
    lons = dataset.variables['longitude']

    latli = numpy.argmin(numpy.abs(lats - latmin))
    latui = numpy.argmin(numpy.abs(lats - latmax))

    lonli = numpy.argmin(numpy.abs(lons - lonmin))
    lonui = numpy.argmin(numpy.abs(lons - lonui))

    dataset = dataset.variables['Rrs'][ : , latli:latui, lonli:lonui]

    datasetred = dataset["Rrs"].sel({"wavelength_3d": 143})
    datasetblue = dataset["Rrs"].sel({"wavelength_3d": 87})
    datasetNDTI = (datasetred - datasetblue)/(datasetred + datasetblue)
    print("NDTI Calculated")
    
    if (ncRasterAverage(latmin, latmax, lonmin, lonmax, datasetNDTI) > 0):
        NDTIAverages.append([dataset.product_name, ncRasterAverage(latmin, latmax, lonmin, lonmax, datasetNDTI)])
    print("Dataset added")

NDTIAverages

PACE_OCI.20250104T202321.L2.OC_AOP.V3_0.NRT.nc


ValueError: dimensions ('number_of_lines', 'pixels_per_line') must have the same length as the number of data dimensions, ndim=0

**GHRSST**

In [11]:
results = earthaccess.search_datasets(provider="POCLOUD")

for item in results: 
    summary = item.summary()
    print(summary["short-name"])

MUR-JPL-L4-GLOB-v4.1
CYGNSS_L1_V3.2
TOPEX_POSEIDON_GDR_F
ASCATB-L2-Coastal
ASCATC-L2-Coastal
OPERA_L3_DSWX-HLS_V1
OSTIA-UKMO-L4-GLOB-REP-v2.0
SWOT_L2_LR_SSH_2.0
CYGNSS_NOAA_L2_SWSP_25KM_V1.2
OSCAR_L4_OC_NRT_V2.0
OSTIA-UKMO-L4-GLOB-v2.0
VIIRS_N20-STAR-L2P-v2.80
ABI_G16-STAR-L3C-v2.70
AVHRRF_MB-STAR-L2P-v2.80
AVHRRF_MC-STAR-L2P-v2.80
CMC0.1deg-CMC-L4-GLOB-v3.0
MERGED_TP_J1_OSTM_OST_CYCLES_V52
MODIS_T-JPL-L2P-v2019.0
MUR25-JPL-L4-GLOB-v04.2
N21-VIIRS-L2P-ACSPO-v2.80
OSCAR_L4_OC_FINAL_V2.0
SWOT_L2_HR_Raster_2.0
VIIRS_NPP-STAR-L2P-v2.80
AMSR2-REMSS-L2P-v8.2
AVHRR_OI-NCEI-L4-GLOB-v2.1
CYGNSS_L1_V2.1
CYGNSS_L2_SURFACE_FLUX_V3.2
DMI_OI-DMI-L4-GLOB-v1.0
GAMSSA_28km-ABOM-L4-GLOB-v01
Geo_Polar_Blended-OSPO-L4-GLOB-v1.0
Geo_Polar_Blended_Night-OSPO-L4-GLOB-v1.0
K10_SST-NAVO-L4-GLOB-v01
MERGED_TP_J1_OSTM_OST_GMSL_ASCII_V52
MW_IR_OI-REMSS-L4-GLOB-v5.0
MW_IR_OI-REMSS-L4-GLOB-v5.1
MW_OI-REMSS-L4-GLOB-v5.0
MW_OI-REMSS-L4-GLOB-v5.1
OISST_HR_NRT-GOS-L4-BLK-v2.0
OISST_HR_NRT-GOS-L4-MED-v2.0
OISST_UHR_NRT-

KeyError: 'FileDistributionInformation'

In [19]:
tspan = timePeriod
bbox = AOI
clouds = (0,50)

results = earthaccess.search_data(
    short_name = "MUR25-JPL-L4-GLOB-v04.2",
    temporal = tspan,
    bounding_box = bbox
)

paths = earthaccess.open(results)
paths 

murSST = []

for file in paths:
    testxr = xr.open_dataset(file)
    murSST.append(testxr)
    print(f'{testxr.id + " " + testxr.start_time} added!')

QUEUEING TASKS | :   0%|          | 0/68 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/68 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/68 [00:00<?, ?it/s]

MUR25-JPL-L4-GLOB-v04.2 20250101T090000Z added!
MUR25-JPL-L4-GLOB-v04.2 20250102T090000Z added!
MUR25-JPL-L4-GLOB-v04.2 20250103T090000Z added!
MUR25-JPL-L4-GLOB-v04.2 20250104T090000Z added!
MUR25-JPL-L4-GLOB-v04.2 20250105T090000Z added!
MUR25-JPL-L4-GLOB-v04.2 20250106T090000Z added!
MUR25-JPL-L4-GLOB-v04.2 20250107T090000Z added!
MUR25-JPL-L4-GLOB-v04.2 20250108T090000Z added!
MUR25-JPL-L4-GLOB-v04.2 20250109T090000Z added!
MUR25-JPL-L4-GLOB-v04.2 20250110T090000Z added!
MUR25-JPL-L4-GLOB-v04.2 20250111T090000Z added!
MUR25-JPL-L4-GLOB-v04.2 20250112T090000Z added!
MUR25-JPL-L4-GLOB-v04.2 20250113T090000Z added!
MUR25-JPL-L4-GLOB-v04.2 20250114T090000Z added!
MUR25-JPL-L4-GLOB-v04.2 20250115T090000Z added!
MUR25-JPL-L4-GLOB-v04.2 20250116T090000Z added!
MUR25-JPL-L4-GLOB-v04.2 20250117T090000Z added!
MUR25-JPL-L4-GLOB-v04.2 20250118T090000Z added!
MUR25-JPL-L4-GLOB-v04.2 20250119T090000Z added!
MUR25-JPL-L4-GLOB-v04.2 20250120T090000Z added!
MUR25-JPL-L4-GLOB-v04.2 20250121T090000Z

In [25]:
SSTaverages = []

for file in murSST:
    meanSST = ncRasterAverage(latmin, latmax, lonmin, lonmax, file["analysed_sst"])
    if (meanSST > 0):
        SSTaverages.append([f'{file.id}_{file.start_time}', meanSST])

SSTaverages

SSTpath = 'WaterQuality/MUR/SST/murSST'
with open(location + SSTpath, 'w') as myFile: 
    wr = csv.writer(myFile)
    for record in SSTaverages:
        wr.writerow(record)

lat: 33.875, lon: -118.625: 287.80199999999996
lat: 33.875, lon: -118.625: 287.596
lat: 33.875, lon: -118.625: 287.397
lat: 33.875, lon: -118.625: 287.74399999999997
lat: 33.875, lon: -118.625: 287.83599999999996
lat: 33.875, lon: -118.625: 287.87899999999996
lat: 33.875, lon: -118.625: 287.74199999999996
lat: 33.875, lon: -118.625: 287.54699999999997
lat: 33.875, lon: -118.625: 287.44399999999996
lat: 33.875, lon: -118.625: 287.394
lat: 33.875, lon: -118.625: 287.42699999999996
lat: 33.875, lon: -118.625: 287.381
lat: 33.875, lon: -118.625: 287.344
lat: 33.875, lon: -118.625: 287.41799999999995
lat: 33.875, lon: -118.625: 287.48499999999996
lat: 33.875, lon: -118.625: 287.48699999999997
lat: 33.875, lon: -118.625: 287.48999999999995
lat: 33.875, lon: -118.625: 287.585
lat: 33.875, lon: -118.625: 287.661
lat: 33.875, lon: -118.625: 287.61499999999995
lat: 33.875, lon: -118.625: 287.572
lat: 33.875, lon: -118.625: 287.549
lat: 33.875, lon: -118.625: 287.599
lat: 33.875, lon: -118.625: 2

**SMAP SSS**

In [35]:
testxr = xr.open_dataset(paths[0], decode_times = False)
testxr.date_created[0 : 10]

'2025-01-10'

In [43]:
tspan = timePeriod
bbox = AOI
clouds = (0,50)

results = earthaccess.search_data(
    short_name = "SMAP_RSS_L2_SSS_V6",
    temporal = tspan, 
    bounding_box = AOI
)

paths = earthaccess.open(results)

SMAP_SSS = []

for file in paths:
    skip = False
    filexr = xr.open_dataset(file, decode_times = False)
    print(filexr.date_created)
    for dataset in SMAP_SSS:
        if (filexr.date_created[0:10] == dataset.date_created[0:10]):
            skip = True
            print("Skipped")
    if (skip == False):
        SMAP_SSS.append(filexr)
        print("Added")

SMAP_SSS

QUEUEING TASKS | :   0%|          | 0/152 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/152 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/152 [00:00<?, ?it/s]

2025-01-10T10:35:54-0800
Added
2025-01-10T10:37:24-0800
Skipped
2025-01-10T10:39:14-0800
Skipped
2025-01-10T10:41:04-0800
Skipped
2025-01-10T10:42:18-0800
Skipped
2025-01-12T12:56:14-0800
Added
2025-02-06T14:23:05-0800
Added
2025-01-14T20:21:21-0800
Added
2025-02-06T14:28:18-0800
Skipped
2025-02-06T14:41:06-0800
Skipped
2025-01-22T21:23:10-0800
Added
2025-01-16T12:47:49-0800
Added
2025-01-17T15:12:22-0800
Added
2025-01-18T11:30:11-0800
Added
2025-01-20T13:30:38-0800
Added
2025-01-21T09:29:39-0800
Added
2025-01-22T11:16:55-0800
Skipped
2025-01-22T11:29:17-0800
Skipped
2025-01-23T10:21:55-0800
Added
2025-02-04T16:42:51-0800
Added
2025-02-04T16:44:50-0800
Skipped
2025-02-04T16:56:41-0800
Skipped
2025-02-04T17:06:14-0800
Skipped
2025-02-04T17:11:25-0800
Skipped
2025-02-04T17:13:17-0800
Skipped
2025-02-04T17:21:50-0800
Skipped
2025-02-04T17:53:27-0800
Skipped
2025-02-04T17:59:19-0800
Skipped
2025-02-04T18:38:14-0800
Skipped
2025-02-04T18:48:05-0800
Skipped
2025-02-04T19:03:43-0800
Skipped
2

[<xarray.Dataset> Size: 1GB
 Dimensions:                 (ydim_grid: 720, xdim_grid: 1560, look: 2,
                              polarization_2: 2, polarization_4: 4,
                              polarization_3: 3, iceflag_components: 3,
                              uncertainty_components: 9)
 Dimensions without coordinates: ydim_grid, xdim_grid, look, polarization_2,
                                 polarization_4, polarization_3,
                                 iceflag_components, uncertainty_components
 Data variables: (12/58)
     time                    (ydim_grid, xdim_grid, look) float64 18MB ...
     cellat                  (ydim_grid, xdim_grid, look) float32 9MB ...
     cellon                  (ydim_grid, xdim_grid, look) float32 9MB ...
     gland                   (ydim_grid, xdim_grid, look) float32 9MB ...
     fland                   (ydim_grid, xdim_grid, look) float32 9MB ...
     gice_est                (ydim_grid, xdim_grid) float32 4MB ...
     ...             

In [117]:
SMAP_SSS[0]["cellon"].sel(ydim_grid = 32, xdim_grid = 100).item(0)

115.14714813232422

In [118]:
SSSaverages = []

for dataset in SMAP_SSS:
    total = 0
    count = 0
    avg = 0

    print(f'{dataset.id}_{dataset.date_created}')

    for y in range (0, 719):
        for x in range(0, 1559):
            lat = dataset["cellat"].sel(ydim_grid = y, xdim_grid = x).item(0)
            if (lat < latmax):
                if (lat > latmin):
                    lon = dataset["cellon"].sel(ydim_grid = y, xdim_grid = x).item(0)
                    if (lon < lonmax):
                        if (lon > lonmin):
                            value = dataset["sss_smap"].sel(ydim_grid = y, xdim_grid = x).item(0)
                            print(f'lon: {lon}, lat: {lat}: {value}')

                            total += value
                            count += 1

                            
    if (count > 0): 
        avg = total/count

    if (avg > 0):
        SSSaverages.append([f'{dataset.id}_{dataset.date_created}', avg])


with open('PFDF_Data/California2025/WaterQuality/SMAP/SSS/smapSSS', 'w') as myFile: 
    wr = csv.writer(myFile)
    for record in SSSaverages:
        wr.writerow(record)

10.5067/SMP60-2SOCS_2025-01-10T10:35:54-0800


KeyboardInterrupt: 